In [2]:
# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName("Capstone Analytics Samples").getOrCreate()

2022-03-01 15:11:15 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
fact_immigration = spark.read.parquet('tables/fact_immigration')
dim_immigrant_person = spark.read.parquet('tables/dim_immigrant_person')
dim_city = spark.read.parquet('tables/dim_city')
dim_time = spark.read.parquet('tables/dim_time')

Create tempory views for the queries

In [4]:
fact_immigration.createOrReplaceTempView("fact_immigration_table")
dim_immigrant_person.createOrReplaceTempView("immigrant_table")
dim_city.createOrReplaceTempView("dim_city_table")
dim_time.createOrReplaceTempView("dim_time_table")

In [29]:
spark.sql("""
    SELECT count(*) FROM fact_immigration_table
""").show()

+--------+
|count(1)|
+--------+
|  219268|
+--------+



Question: Do immigrants generally flock to cities with generally more immigrants?

To answer this question, we combine the city demographics data with the immigration fact data and order it by foreign_borns.

In [37]:
spark.sql("""
    SELECT *
    FROM
        (SELECT DISTINCT
            f.i94port, 
            COUNT(f.cicid) AS immigrants
        FROM fact_immigration_table f
        GROUP BY f.i94port
        ORDER BY immigrants DESC
        LIMIT 10
        ) AS immigrants_by_port
    JOIN dim_city_table c
        ON immigrants_by_port.i94port = c.city_code
    ORDER BY c.foreign_born DESC
""").dropDuplicates().show()

+-------+----------+---------------+----------+----------+---------------+----------------+------------+---------------------+----------+---------+
|i94port|immigrants|      city_name|     state|median_age|male_population|total_population|foreign_born|average_householdsize|state_code|city_code|
+-------+----------+---------------+----------+----------+---------------+----------------+------------+---------------------+----------+---------+
|    NYC|     32779|       New York|  New York|      36.0|        4081698|         4468707|     8550405|             156961.0|   3212500|      NYC|
|    LOS|     20983|    Los Angeles|California|      35.0|        1958998|         2012898|     3971896|              85417.0|   1485425|      LOS|
|    CHI|     11823|        Chicago|  Illinois|      34.2|        1320015|         1400541|     2720556|              72042.0|    573463|      CHI|
|    HOU|      7305|        Houston|     Texas|      32.6|        1149686|         1148942|     2298628|        